In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from IPython.display import clear_output

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sip=np.array(payload.iloc[:,0])

df=pd.DataFrame()

#date stuff
today = datetime.date.today()
friday = (today + datetime.timedelta( (4-today.weekday()) % 7 ))
start_date="2010-01-01"
end_date=today
#yesterday = today - datetime.timedelta(days=1)

#days till expriy
dow=today.weekday()
if dow < 5: 
    dte=5-dow
else: 
    dte=12-dow


#for i in range(0,len(sip)):
def st_rsi():
    df_st_rsi=pd.DataFrame()
    for i in range(0,len(sip)):
    #for i in range(0,5):
        stk=sip[i]
        dfw = yf.download(stk, start_date, end_date,interval='1wk').dropna()
        dfm = yf.download(stk, start_date, end_date,interval='1mo').dropna()
        #dfd = yf.download(stk, yesterday, today,interval='1h').dropna()
        
        stoch_rsi_w=StochRSIIndicator(close=dfw.Close,window=14,smooth1=3, smooth2=3)
        stoch_rsi_m=StochRSIIndicator(close=dfm.Close,window=14,smooth1=3, smooth2=3)
        
        efiw=ForceIndexIndicator(close=dfw.Close,volume=dfw.Volume,window=14,fillna=True)
        dfw['efi_w']=efiw.force_index()
    
        rsiw=RSIIndicator(close=dfw.efi_w,window=14,fillna=True)
        dfw['efi_rsi_w']=rsiw.rsi()
        
        efim=ForceIndexIndicator(close=dfm.Close,volume=dfm.Volume,window=14,fillna=True)
        dfm['efi_m']=efim.force_index()
    
        rsim=RSIIndicator(close=dfm.efi_m,window=14,fillna=True)
        dfm['efi_rsi_m']=rsim.rsi()
        
        
        dfw["kw"]=stoch_rsi_w.stochrsi_k()
        
        dfm["km"]=stoch_rsi_m.stochrsi_k()
        dfw["Chng"]=(dfw.Close-dfw.Open)/dfw.Open
        STD=(np.std(dfw.Chng))
       
        lw=len(dfw.iloc[:,0])
        lm=len(dfm.iloc[:,0])
        #ld=len(dfd.iloc[:,0])
    
        last=dfw['Close'][lw-1]
        kw=dfw['kw'][lw-1]
        km=dfm['km'][lm-1]
            
        efw=dfw['efi_w'][lw-1]
        ef_rsi_w=(dfw['efi_rsi_w'][lw-1])/100
    
        efm=dfm['efi_m'][lm-1]
        ef_rsi_m=dfm['efi_rsi_m'][lm-1]/100
        
        #print(dfm(tail())

        #print(kw,km)
        temp=pd.Series([stk,last,STD,kw,km,ef_rsi_w,ef_rsi_m,efw,efm])
        #print(temp)
        df_st_rsi=df_st_rsi.append(temp,ignore_index=True)
        print(i ,'/', len(sip))
        clear_output(wait=True)
    df_st_rsi.columns=['stk','last','STD','kw','km','ef_rsi_w','ef_rsi_m','efw','efm']
    return(df_st_rsi)

def get_call():
    df_call=pd.DataFrame()
    #for i in range(0,len(sip)):
    for i in range(0,50):
        try:
            stk=sip[i]
            tick= yf.Ticker(stk)
            ch=tick.option_chain(date=str(friday)).calls
            ch=ch[ch.inTheMoney==False]
            strike=ch.iloc[0,2]
            premium=ch.iloc[0,3] 
            
            
            temp=pd.Series([stk,strike,premium])
            df_call=df_call.append(temp,ignore_index=True)
        except:
            pass
        print(i ,'/', len(sip))
        clear_output(wait=True)
    df_call.columns=['stk','strike','premium']
    df_call['expiry_date']=friday
    df_call['days_to_expiry']=dte
    return(df_call)


def get_earns():
    df_earn=pd.DataFrame()
    for i in range(0,len(sip)):
        tick= yf.Ticker(sip[i])
        try:
            earns=pd.Timestamp(tick.calendar.iloc[0,0])
        except:
            pass      
        temp=pd.Series([sip[i],earns])
        df_earn=df_earn.append(temp,ignore_index=True)
        print(i ,'/', len(sip))
        clear_output(wait=True)
    df_earn.columns=['stk','earnings_date']
    df_earn['e_pass']=np.where(df_earn['earnings_date']<=pd.Timestamp(friday),1,0)
    return(df_earn)


   stk       earnings_date  e_pass
0  MMM 2022-01-24 13:30:00       0
1  ABT 2022-01-25 13:30:00       0


In [2]:
f_cll=get_call()
f_cll=f_cll[f_cll['strike']<200]
print(f_cll)
sip=f_cll['stk'].to_numpy()

     stk  strike  premium expiry_date  days_to_expiry
1    AMD   128.0     1.77  2021-11-05               4
2    AFL    55.0     0.38  2021-11-05               4
6    AAL    20.0     0.22  2021-11-05               4
9    ADI   177.5     1.25  2021-11-05               4
10   APA    27.5     0.63  2021-11-05               4
11  AAPL   152.5     0.63  2021-11-05               4
12   ADM    64.0     0.23  2021-11-05               4


463 / 505


In [4]:
f_rsi=st_rsi()
f_rsi=f_rsi[f_rsi['kw']>.7 | ]
print(f_rsi)
sip=f_rsi['stk'].to_numpy()

   stk        last       STD        kw        km  ef_rsi_w  ef_rsi_m  \
0  AMD  125.230003  0.071998  0.924802  0.615757  0.527350  0.537594   
4  APA   27.170000  0.071311  1.000000  1.000000  0.601235  0.575813   
6  ADM   63.610001  0.032428  0.887091  0.508249  0.491974  0.536855   

            efw           efm  
0  5.364089e+08  4.370270e+09  
4  3.505758e+07  2.088656e+08  
6  2.108421e+06  4.621759e+07  


In [ ]:

f_ern=get_earns()
f_ern=f_ern[f_ern['e_pass']==0]
print(f_ern)
#sip=f_ern['stk'].to_numpy()

In [ ]:
ff=f_cll.set_index('stk').join(f_rsi.set_index('stk')).join(f_ern.set_index('stk'))
#ff=ff.join(f4.set_index('stk'))

ff['yield']=ff['premium']/ff['last']
ff['gap_p']=(ff['strike']-ff['last'])/ff['last']
ff['final_premium']=ff['gap_p']*ff['yield']/ff['STD']

print(ff)
#ff.to_csv('trade_matrix.csv')